In [1]:
import sys
import os
source_dir = os.path.expanduser("~/Source/Turing/pyNetEMD/")
data_dir = os.path.expanduser("~/")
sys.path.append(source_dir)
import pyximport
pyximport.install(reload_support=True)
import cyemdORB4 as cy
from scipy.optimize import minimize_scalar

In [2]:
# Mass and variance normalised histogram example (initial integer histogram becomes 
# real-valued locations and masses on non-aligned grids)
# Smoothed area between cumulative functions should be very close to 0.534
reload(cy)
#from IPython.core.debugger import Tracer
counts1_test = [1, 1, 1, 1, 3, 3, 4, 4]
counts2_test = [0, 2, 2, 2, 2, 4, 5, 5]
dis1_test = cy.dis(counts1_test)
dis2_test = cy.dis(counts2_test)
dis1_rescaled_test = cy.rescale(dis1_test)
dis2_rescaled_test = cy.rescale(dis2_test)
shift1 = 0.9028494 - dis1_rescaled_test[0][0]; scale1 = 1
dis1_rescaled_shifted_test = [[item[0]+shift1, item[1]+shift1, item[2]*scale1] for item in dis1_rescaled_test]
shift2 = 0.7675186 - dis2_rescaled_test[0][0]; scale2 = 1
dis2_rescaled_shifted_test = [[item[0]+shift2, item[1]+shift2, item[2]*scale2] for item in dis2_rescaled_test]
dis1_cum_test = cy.cum(dis1_rescaled_shifted_test)
dis2_cum_test = cy.cum(dis2_rescaled_shifted_test)
#ipdb.set_trace()

actual_area_test = cy.pyKSAt(dis1_cum_test, dis2_cum_test, 0)
print(dis1_cum_test)
print(dis2_cum_test)
print(actual_area_test)

[[ 0.90284938  0.        ]
 [ 1.67264974  0.5       ]
 [ 2.44245005  0.5       ]
 [ 3.21225047  0.75      ]
 [ 3.9820509   1.        ]]
[[ 0.76751858  0.        ]
 [ 1.37751293  0.125     ]
 [ 1.9875071   0.125     ]
 [ 2.59750152  0.625     ]
 [ 3.20749569  0.625     ]
 [ 3.8174901   0.75      ]
 [ 4.42748451  1.        ]]
0.533898293972


In [3]:
min_emd, offset = cy.KSAPF(dis1_cum_test, dis2_cum_test)
[min_emd, offset]

[0.26814886927604675, 0.54629142203876391]

In [5]:
# Confirm smoothed histogram formulae for E[X] and E[X^2] (integrating over bin width)
from sympy import *
x1 = symbols('x1')
x2 = symbols('x2')
x = symbols('x')
exp_x = simplify(integrate((1/(x2-x1))*(x), (x, x1, x2)))
exp_x2 = simplify(integrate((1/(x2-x1))*(x**2), (x, x1, x2)))
print("E[X] = {0}".format(exp_x))
print("E[X^2] = {1}".format(exp_x, exp_x2))

E[X] = x1/2 + x2/2
E[X^2] = x1**2/3 + x1*x2/3 + x2**2/3


In [11]:
# Determine difference between smoothed and unsmoothed histogram values for E[X^2]
from sympy import *
x_lower = symbols('x_lower')
x_upper = symbols('x_upper')
x_centre = symbols('x_centre')
bin_width = symbols('bin_width')
smoothed_exp_x2 = (x_lower**2/3 + x_upper*x_lower/3 + x_upper**2/3)
smoothed_exp_x2 = simplify(smoothed_exp_x2.subs(x_lower, x_centre - bin_width/2).subs(x_upper, x_centre + bin_width/2))
unsmoothed_exp_x2 = expand(((x_lower + x_upper)/2)**2)
unsmoothed_exp_x2 = simplify(unsmoothed_exp_x2.subs(x_lower, x_centre - bin_width/2).subs(x_upper, x_centre + bin_width/2))
print("Smoothed E[X^2] = {0}".format(smoothed_exp_x2))
print("Unsmoothed E[X^2] = {0}".format(unsmoothed_exp_x2))

Smoothed E[X^2] = bin_width**2/12 + x_centre**2
Unsmoothed E[X^2] = x_centre**2
